In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
import os

In [2]:
dataPath = "datasets/housing"
dataFileName = "housing.csv"
def loadData(dataPath,dataFileName):
    fullPath = os.path.join(dataPath,dataFileName)
    return pd.read_csv(fullPath)

housing = loadData(dataPath,dataFileName)

housing['income_cat'] = np.ceil(housing['median_income']/1.5)
housing['income_cat'].where(housing['income_cat']<5,5,inplace = True)

housing_labels = housing["median_house_value"].copy()
housing = housing.drop("median_house_value", axis=1)

In [3]:
from sklearn.model_selection import StratifiedShuffleSplit

def train_test_stratSplit(data,labels):
    split = StratifiedShuffleSplit(n_splits = 1,train_size = 0.8,random_state = 42)
    for trainIdx,testIdx in split.split(data,data['income_cat']):
        trainDataX = data.loc[trainIdx]
        testDataX = data.loc[testIdx]
        trainDataY = labels.loc[trainIdx]
        testDataY = labels.loc[testIdx]
    return (trainDataX,trainDataY,testDataX,testDataY)

trainDataX,trainDataY,testDataX,testDataY = train_test_stratSplit(housing,housing_labels)

In [4]:
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import FeatureUnion
from sklearn.preprocessing import OneHotEncoder
from sklearn.base import TransformerMixin,BaseEstimator

In [5]:
rooms_ix, bedrooms_ix, population_ix, household_ix = 3, 4, 5, 6

class AttributesAdder:
    def __init__(self):
        pass
    def fit(self,data,y=None):
        return self
    def transform(self,data,y=None):
        Rooms_Per_Household = data[:,rooms_ix]/data[:,household_ix]
        Population_Per_Household = data[:,population_ix]/data[:,household_ix]
        Bedrooms_Per_Household = data[:,bedrooms_ix]/data[:,household_ix]
        return np.c_[data, Rooms_Per_Household, Population_Per_Household,Bedrooms_Per_Household]
    
class DataFrameSelector:
    def __init__(self,AttributeNames):
        self.AttributeNames = AttributeNames
    def fit(self,data,y=None):
        return self
    def transform(self,data,y=None):
        return data[self.AttributeNames].values
        


In [6]:
numericalAttribute = list(trainDataX)
numericalAttribute.remove("ocean_proximity")
categoricalAttribute = ["ocean_proximity"]

numerical_pipeline = Pipeline([('selector',DataFrameSelector(numericalAttribute)),
                                ('imputer',SimpleImputer(strategy = "median")),
                                ('Attribute_Adder',AttributesAdder()),
                                ('std_scaler',StandardScaler()),
                              ])

categorical_pipeline = Pipeline([('selector',DataFrameSelector(categoricalAttribute)),
                                 ('one_hot_encoder', OneHotEncoder(sparse=False)),
                                ])

full_pipeline = FeatureUnion(transformer_list=[("numerical_pipeline",numerical_pipeline),
                                               ("categorical_pipeline",categorical_pipeline),
                                              ])

In [7]:
trainDataX = full_pipeline.fit_transform(trainDataX)
testDataX = full_pipeline.fit_transform(testDataX)

In [8]:
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()
lin_reg.fit(trainDataX, trainDataY)


from sklearn.tree import DecisionTreeRegressor
tree_reg = DecisionTreeRegressor()
tree_reg.fit(trainDataX, trainDataY)

DecisionTreeRegressor()

In [9]:
from sklearn.metrics import mean_squared_error

In [15]:
DataPred_lin_reg = lin_reg.predict(testDataX)
mselin = mean_squared_error(testDataY,DataPred_lin_reg)
np.sqrt(mselin)

67266.54562979023

In [16]:
DataPred_decTree = tree_reg.predict(testDataX)
msetree = mean_squared_error(testDataY,DataPred_decTree)
np.sqrt(msetree)

104381.34348652349